# Integrazione dei dati Netflix con dati Wikipedia

I dati Netflix si riferiscono al periodo tra il 1 Dicembre 2020 e il 31 Marzo 2021.

In [1]:
import pandas as pd
import re

Lettura dei file con i dati da flixpatrol in csv

In [2]:
netflix = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/netflix_dic-mar2021.csv", index_col = 0)
top_ten = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_dic-mar2021.csv", index_col = 0)
top_film = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_film_dic-mar2021.csv", index_col = 0)
top_serie = pd.read_csv("Data_flixpatrol/Dati_netflix_dicembre-aprile/top_ten_serie_dic-mar2021.csv", index_col = 0)

In [3]:
netflix.head(3)

,result,id,name,url,imdb,tmdb,premiere,type_id,type,country_id,...,streaming,from,to,ranking,ranking_last,value,value_last,value_total,countries,days
0,1.0,97483.0,The Queen&#39;s Gambit,https://flixpatrol.com/title/the-queens-gambit,10048342,87739,2020.0,2.0,TV,4672.0,...,656.0,2020-12-01,2020-12-01,1.0,0.0,10.0,0.0,0.0,0.0,0.0
1,2.0,72764.0,Virgin River,https://flixpatrol.com/title/virgin-river,9077530,88324,2019.0,2.0,TV,4672.0,...,656.0,2020-12-01,2020-12-01,2.0,0.0,9.0,0.0,0.0,0.0,0.0
2,3.0,99577.0,The Beast,https://flixpatrol.com/title/the-beast-2020,11499506,654905,2020.0,1.0,Movie,93.0,...,656.0,2020-12-01,2020-12-01,3.0,0.0,8.0,0.0,0.0,0.0,0.0


### Estraggo,correggo i titoli e li ordino in ordine alfabetico

Alcuni titoli presentano delle imperfezioni perché contengono un apostrofo.

In [4]:
titoli_raw = list(set(netflix['name']))  # titoli così come sono nel database

titoli = titoli_raw.copy() # titoli che saranno corretti

In [5]:
len(titoli_raw)

208

__In tutto ho 208 titoli.__
Alcuni titoli nel database originario (contenuti in _titoli_raw_) sono scritti male, quindi li correggo. _titoli_raw_ conterrà i titoli originari, mentre _titoli_ i titoli corretti.

In [6]:
# Stampo i titoli scritti male (= con un apostrofo)

for t in titoli_raw:
    occ = re.compile('^.*&\#39;.*$')
    
    if occ.match(t):
        print(t)

So My Grandma&#39;s a Lesbian!
Ma Rainey&#39;s Black Bottom
To All the Boys I&#39;ve Loved Before
Don&#39;t Listen
I&#39;m Not a Killer
Finding &#39;Ohana
Dawson&#39;s Creek
The Queen&#39;s Gambit
A Dog&#39;s Way Home


In [7]:
# Correggo (in titoli) i titoli scritti male

for t in titoli:
    
    occ = re.compile('(^.*)(\&\#39;)(.*$)')
    found = occ.match(t)
    
    if found:
        titoli.remove(t)
        pulita = re.sub(r'(\&\#39;)',"\'" ,t)
        titoli.append(pulita)

In [8]:
# Ordino in ordine alfabetico

titoli = sorted(titoli)
titoli_raw = sorted(titoli_raw)

Converto la colonna degli anni dal db _netflix_ in intero.

In [9]:
netflix['premiere'] = netflix.premiere.astype(int)

## Integro dati top ten generale con Wikipedia

Utilizzo la libreria wikipediaapi, oltre alla libreria BeautifulSoup per lo scraping

In [10]:
import wikipedia
import wikipediaapi

In [11]:
# Imposto wikipedia inglese
wiki_wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI)

# Il formato HTML non dà comunque le tabelle

In [12]:
import requests
from bs4 import BeautifulSoup
import json

Creo dizionario con per ogni titolo la propria pagina Wikipedia di riferimento, fornita da wikipediaapi.
A tale scopo creo la funzione wikipages: essa cerca in WIkipedia inglese ogni titolo, dopo aver controllato nel dataframe se esso è una serie tv o un film. 
Lo scopo della funzione è trovare tramite la libreria più titoli possibile nella lista; infatti dopo aver cercato il titolo puro, nel caso la pagina non esista, lo si ricerca in Wikipedia aggungendogli "( film)" e l'anno in cui è uscito se il titolo è un film, mentre se è una serie tv (o una miniserie) lo si ricerca aggiungendo "(TV series)" o "(miniseries)". Queste apposizioni sono in genere gli URL della Wikipedia inglese.

Per alcuni titoli non viene trovata nessuna pagina: nel dizionario sarà loro assegnata la stringa "0".

In [13]:
# db è il database da usare (in questo caso netflix), titoli la lista dei titoli corretti, titoli_raw la lista dei titoli non corretti

def wikipages(db, titoli, titoli_raw):
    
    diz = {}
    limit = len(titoli)

    for i in range(limit):

        t = titoli[i]  # titolo corrente
        t_raw = titoli_raw[i]  #titolo corrente senza caratteri corretti

        tipo = list(db[db['name'] == t_raw]['type'])[0] # è una serie tv (TV) o un film (Movie)?

        parola_chiave = t

        # Cerco i film
        if tipo == "Movie": 
            parola_chiave = parola_chiave +" ("+ str(list(db[db['name'] == t_raw]['premiere'])[0]) +" film)"

            page = wiki_wiki.page(parola_chiave)

            if page.exists():
                diz[t] = page

            else:
                parola_chiave = t + " (film)"
                page = page = wiki_wiki.page(parola_chiave)

                if page.exists():
                    diz[t] = page
                else:
                    page = wiki_wiki.page(t)

                    if page.exists():
                        diz[t] = page
                    else:
                        diz[t] = "0"   # se non trovo la pagina assegno al titolo la stringa "0"

        # Cerco le serie tv
        elif tipo == "TV":
            
            parola_chiave = parola_chiave + " ("+ str(list(db[db['name'] == t_raw]['premiere'])[0]) +" TV series)"
            page = wiki_wiki.page(parola_chiave)

            if page.exists():
                diz[t] = page

            else:
                parola_chiave = t + " (TV series)"
                page = wiki_wiki.page(parola_chiave)
                
                if page.exists():
                    diz[t] = page
                    
                else:
                    parola_chiave = t + " (miniseries)"
                    page = wiki_wiki.page(parola_chiave)

                    if page.exists():
                        diz[t] = page

                    else:
                        page = wiki_wiki.page(t)

                        if page.exists():
                            diz[t] = page

                        else:
                            diz[t] = "0"
        else:
            diz[t] = "0"
    
    return diz

In [14]:
diz = wikipages(netflix, titoli, titoli_raw)

In [15]:
diz

{'365 Days': 365 Days (2020 film) (id: 64226248, ns: 0),
 '5 Is the Perfect Number': 5 Is the Perfect Number (id: 61358069, ns: 0),
 'A California Christmas': A California Christmas (id: 66054729, ns: 0),
 "A Dog's Way Home": A Dog's Way Home (id: 57758112, ns: 0),
 'A Star is Born': A Star Is Born (id: 61533, ns: 0),
 'A Week Away': A Week Away (id: 66054772, ns: 0),
 'Addicted': Addicted (2014 film) (id: 40593969, ns: 0),
 'Age of Samurai: Battle for Japan': Age of Samurai: Battle for Japan (id: 63068384, ns: 0),
 'Agents of S.H.I.E.L.D.': Agents of S.H.I.E.L.D. (id: 36860986, ns: 0),
 'Alice in Borderland': Alice in Borderland (TV series) (id: 66174045, ns: 0),
 'Alien Worlds': Alien Worlds (TV series) (id: 65951627, ns: 0),
 'All My Friends Are Dead': All My Friends Are Dead (id: 27962354, ns: 0),
 'Animals on the Loose: A You vs. Wild Interactive Movie': '0',
 'As Above, So Below': As Above, So Below (film) (id: 42141285, ns: 0),
 'Asphalt Burning': '0',
 'Ava': Ava (2020 film) (i

In [16]:
len(diz)

208

Creo la lista _titoli_mancanti_, in cui metto i titoli il cui url non è stato ancora trovato (identificati dalla stringa "0"), e dopo aggiungerò quelli senza infobox standard quando si passserà allo scraping.

In [17]:
# Per alcuni trova la pagina desambiaguata : li aggiungo ai titoli mancanti

diz["The Champion"] = "0"
diz['Tiramisu'] = "0"
diz["Coven"] = "0"

In [18]:
titoli_mancanti = [] 

for t in titoli:
    if diz[t] == "0":
        titoli_mancanti.append(t)

In [19]:
len(titoli_mancanti)

18

Ho 18 titoli mancanti su 208

In [20]:
titoli_mancanti

['Animals on the Loose: A You vs. Wild Interactive Movie',
 'Asphalt Burning',
 'Biggie: I Got a Story to Tell',
 'Coven',
 "Don't Listen",
 'Full Out 2: You Got This!',
 'Grizzy & the Lemmings',
 "I'm Not a Killer",
 'Il Traditore',
 'Il signor Diavolo',
 'La prima pietra',
 'Midnight at the Magnolia',
 'Mister Felicita',
 'Modalita aereo',
 'SanPa Sins of the Savior',
 "So My Grandma's a Lesbian!",
 'The Champion',
 'Tiramisu']

## Integrazione con dati da Wikipedia usando WikipediaApi e BeautifulSoup

Uso BeautifulSoup e gli url forniti dalla libreria WikipediaApi per creare un dizionario (__diz_tab__) in cui ad ogni titolo è associata l'infobox laterale della propria pagina wikipedia.

In [22]:
diz_tab = {}

for t in diz:
    
    if diz[t] != "0":
        
        # Scraping
        response = requests.get(diz[t].fullurl)
        html_page = response.text
        soup = BeautifulSoup(html_page, "html.parser")
        
        # Leggo la tabella a lato
        tabella = soup.find("table", class_="infobox vevent")
        
        if tabella:
            df = pd.read_html(str(tabella), skiprows = 1)[0]
            df.columns = ["param", "title"]
            
            diz_tab[t] = df  # nel dizionario a ogni titolo corrisponde un DataFrame
            
        # Se non trovo la tabella aggiungo il titolo a titoli_mancanti
        else:
            titoli_mancanti.append(t)

Per 8 titoli non ha trovato la tabella: ora i titoli di cui ho i dati forniti dalla tabella wikipedia sono 182 mentre i titoli di cui non ho i dati sono 26.

In [23]:
print(len(diz_tab))
print(len(titoli_mancanti))

182
26


### Devo creare due nuovi DataFrame (uno per i film e uno per le serie tv) con: 
* Il paese d'origine;
* La casa di produzione;
* La distribuzione (_distributor_ o _distributed by_);
* Il cast (come stringa unica);
* L'ideatore (_created by_, per serie tv, _written by_ o _screenplay by_ per i film)
* La durata (per i film) o la durata per episodio (serie tv);
* Il genere (solo per serie tv); 
* Il numero delle stagioni (solo per serie tv);
* Il numero di episodi (solo per serie tv);
* Il regista (solo per i film),
* Il budget (solo per i film),
* Il box office (solo per i film).

Creo _tipo_titoli_ : un dizionario ausiliario che per ogni titolo registra se è un film o una serie tv (per tutti i titoli inclusi quelli mancanti).

In [24]:
tipo_titoli = {}
limit = len(titoli)

for i in range(limit):
    
    # Uso la lista titoli_raw perché alcuni titoli sono scritti in modo diverso nel db originale
    tipo_titoli[titoli[i]] = list(netflix[netflix['name'] == titoli_raw[i]]['type'])[0]

In [25]:
len(tipo_titoli)

208

Conto quante serie tv e quanti film ci sono : ho più film (140) che serie tv (68).

In [26]:
n = 0

for t in tipo_titoli:
    if tipo_titoli[t] == 'TV':
        n = n+1
        
print(n)   # numero serie tv
print(len(tipo_titoli)) # titoli totali

68
208


## Creazione Dataframe

La prima è una funzione ausiliaria per le funzioni successive, che servono a creare i due dataframe.

In [27]:
def aggiungi_valore(df, Dizionario, stringa, categoria):
    
    result = df[df['param'] == stringa]["title"]
    
    if len(result) != 0:
        Dizionario[categoria].append(result.iloc[0])
    
    else:
        Dizionario[categoria].append('NaN')

Funzione per creare il database delle serie tv (_Serie tv_)

In [28]:
def crea_df_serie(Serie_tv, t, df):    # df sarà diz_tab[t], il dataframe che considero volta per volta
    
    Serie_tv['Titolo'].append(t)
    
    # 1. Paese di origine
    result = df[df['param'] == 'Country of origin']["title"]  
    
    if len(result) != 0:
        Serie_tv['Paese'].append(result.iloc[0])
    else:      
        aggiungi_valore(df, Serie_tv, 'Country', 'Paese')
   
    # 2. Produzione
    result = df[df['param'] == 'Productioncompany']['title']
    
    if len(result) != 0:
        Serie_tv['Produzione'].append(result.iloc[0])
    else:
        result = df[df['param'] == 'Productioncompanies']['title']
        
        if len(result) != 0:
            Serie_tv['Produzione'].append(result.iloc[0])
        else:
            result = df[df['param'] == 'Production company']['title']
            
            if len(result) != 0:
                Serie_tv['Produzione'].append(result.iloc[0]) 
            else:
                aggiungi_valore(df, Serie_tv, 'Production companies', 'Produzione')
    
    # 3. Distribuzione
    aggiungi_valore(df, Serie_tv, 'Distributor', 'Distribuzione')

    # 4. Cast
    aggiungi_valore(df, Serie_tv, 'Starring', 'Cast')
    
    # 5. Creatore
    aggiungi_valore(df, Serie_tv, 'Created by', 'Creatore')  # aggiungere developed by?
    
    # 6. Durata per episodio
    aggiungi_valore(df, Serie_tv, 'Running time', 'Durata_episodio')
        
    # 7. Genere
    aggiungi_valore(df, Serie_tv, 'Genre', 'Genere')
    
    # 8. Numero di stagioni
    aggiungi_valore(df, Serie_tv, 'No. of seasons', 'N_stagioni')
        
    # 9. Numero di episodi
    aggiungi_valore(df, Serie_tv, 'No. of episodes', 'N_episodi')

Funzione per creare il database dei film (Film)

In [29]:
def crea_df_film(Film, t, df):
    
    Film['Titolo'].append(t)

    # 1. Paese
    result = df[df['param'] == 'Country']["title"]
    
    if len(result) != 0:
        Film['Paese'].append(result.iloc[0])
    else:
        aggiungi_valore(df, Film, 'Countries', 'Paese')
            
    # 2. Produzione
    result = df[df['param'] == 'Productioncompanies']["title"]
    
    if len(result) != 0:
        Film['Produzione'].append(result.iloc[0])
    else:
        result = df[df['param'] == 'Productioncompany']["title"]
        
        if len(result) != 0:
            Film['Produzione'].append(result.iloc[0])
        else:
            result = df[df['param'] == 'Production company']['title']
            
            if len(result) != 0:
                Film['Produzione'].append(result.iloc[0])
            else:
                aggiungi_valore(df, Film, 'Production companies', 'Produzione')
            
    # 3. Distribuzione
    aggiungi_valore(df, Film, 'Distributed by', 'Distribuzione')
    
    # 4. Cast
    aggiungi_valore(df, Film, 'Starring', 'Cast')
    
    # 5. Creatore
    result = df[df['param'] == 'Written by']["title"]
    
    if len(result) != 0:
        Film['Creatore'].append(result.iloc[0])
    else:
        aggiungi_valore(df, Film, 'Screenplay by', 'Creatore')
    
    # 6. Durata
    aggiungi_valore(df, Film, 'Running time', 'Durata')
    
    # 7. Regista
    aggiungi_valore(df, Film, 'Directed by', 'Regista')
    
    # 8. Budget
    aggiungi_valore(df, Film, 'Budget', 'Budget')
    
    # 9. Box office
    aggiungi_valore(df, Film, 'Box office', 'Box_office')

Funzione generale che richiama le due precedenti.

In [30]:
# Database per i film
Film = {'Titolo' : [], 'Paese' : [], 'Produzione' : [], 'Distribuzione' : [], 'Cast' : [], 'Creatore' : [],
        'Durata' : [], 'Regista' : [], 'Budget' : [], 'Box_office' : []}

# Database per le serie tv
Serie_tv = {'Titolo' : [], 'Paese' : [], 'Produzione' : [], 'Distribuzione' : [], 'Cast' : [], 'Creatore' : [], 
            'Durata_episodio' : [], 'Genere' : [], 'N_stagioni' : [], 'N_episodi' : []}

for t in diz_tab:
    
    tipo = tipo_titoli[t]     # il tipo: serie tv o film; uso il dizionario ausiliario

    if tipo == "Movie":
        crea_df_film(Film, t, diz_tab[t])
    else:
        crea_df_serie(Serie_tv,t,diz_tab[t])

## DataFrame risultanti.

In [31]:
# Dataframe per serie tv
df_serie_tv = pd.DataFrame(data = Serie_tv)

In [32]:
# Dataframe per i film
df_film = pd.DataFrame(data = Film)

__Osservazione:__ il db delle serie tv ha 64 righe, mentre quello dei film ne ha 118 (in tutto 182, ossia la lunghezza di _diz_tab_). Mancano i 26 _titoli_mancanti_.

In [33]:
print(len(df_serie_tv))
print(len(df_film))

64
118


## Usando i metodi delle due librerie prendo alcune informazioni dalle pagine dei titoli.
Arricchisco i due dataframe aggiungendo:
* Sommario/Synopsis
* Trama
* Recensioni
* Controversie (se ci sono)
* Premi
* Cast (in forma di testo)
* Produzione (in forma di testo)

Aggiungo quattro nuove colonne al __dataframe delle serie__: sommario, production, reception e cast.

In [34]:
summary = []
cast = []
production = []
reception = []

for t in diz:
    if t not in titoli_mancanti:
        if tipo_titoli[t] == "TV":    # tipo_titoli non ha i titoli mancanti
            
            p = diz[t]
            
            # Riassunto
            summary.append(p.summary)
            
            # Cast and characters
            cast.append(p.section_by_title("Cast and characters"))
            
            # Production
            production.append(p.section_by_title("Production"))
            
            # Reception
            reception.append(p.section_by_title("Reception"))

In [35]:
# Aggiungo le nuove colonne a df_serie_tv

df_serie_tv['Sommario'] = summary
df_serie_tv['Cast_testo'] = cast
df_serie_tv['Produzione_testo'] = production
df_serie_tv['Critica'] = reception

In [36]:
df_serie_tv.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata_episodio,Genere,N_stagioni,N_episodi,Sommario,Cast_testo,Produzione_testo,Critica
0,Age of Samurai: Battle for Japan,NaN,Netflix,NaN,Masayoshi Haneda Masami Kosaka Hideaki Itō Hay...,NaN,43–45 minutes,NaN,1,6,Age of Samurai: Battle for Japan is an America...,None,None,None
1,Agents of S.H.I.E.L.D.,United States,ABC Studios Marvel Television Mutant Enemy Pro...,Walt Disney Television,Clark Gregg Ming-Na Wen Brett Dalton Chloe Ben...,Joss Whedon Jed Whedon Maurissa Tancharoen,41–44 minutes,Action Drama Science fiction Superhero,7,136 (list of episodes),Marvel's Agents of S.H.I.E.L.D. is an American...,Section: Cast and characters (1):\nClark Gregg...,Section: Production (1):\n\nSubsections (8):\n...,Section: Reception (1):\n\nSubsections (4):\nS...
2,Alice in Borderland,Japan,Robot Communications Inc.,Netflix,Kento Yamazaki Tao Tsuchiya Yūki Morinaga Keit...,NaN,41–52 minutes,Science fiction[1] Suspense-thriller[2] Drama[3],1,8 (list of episodes),"Alice in Borderland (Japanese: 今際の国のアリス, Hepbu...",Section: Cast and characters (1):\n\nSubsectio...,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\n\nSubsections (2):\nS...


Aggiungo nuove colonne anche al __dataframe dei film__: summary, plot, cast, production, reception.

In [37]:
summary = []
plot = []
cast = []
production = []
reception = []

for t in diz:
    if t not in titoli_mancanti:
        
        if tipo_titoli[t] == "Movie":
            p = diz[t]

            # Riassunto
            summary.append(p.summary)

            # Trama
            plot.append(p.section_by_title("Plot"))

            # Cast and characters
            cast.append(p.section_by_title("Cast"))

            # Production
            production.append(p.section_by_title("Production"))

            # Reception
            reception.append(p.section_by_title("Reception"))

In [38]:
# Aggiungo nuove colonne a df_film

df_film['Sommario'] = summary
df_film['Trama'] = plot
df_film['Cast_testo'] = cast
df_film['Produzione_testo'] = production
df_film['Critica'] = reception

In [39]:
df_film.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata,Regista,Budget,Box_office,Sommario,Trama,Cast_testo,Produzione_testo,Critica
0,365 Days,Poland,Ekipa Sp. z o.o. Future Space Next Film TVN,Next Film,Anna-Maria Sieklucka Michele Morrone Bronisław...,Tomasz Klimala Barbara Białowąs Tomasz Mandes ...,114 minutes,Barbara Białowąs Tomasz Mandes,NaN,$9.5 million[1],365 Days (Polish: 365 Dni) is a 2020 Polish er...,Section: Plot (1):\nAfter a meeting between th...,Section: Cast (1):\nMichele Morrone as Don Mas...,Section: Production (1):\nThe film scenes were...,Section: Reception (1):\nThe film made the top...
1,5 Is the Perfect Number,Italy,Rai Cinema,01 Distribution,Toni ServilloValeria GolinoCarlo BuccirossoIai...,Igort,100 minutes,Igort,NaN,NaN,5 Is the Perfect Number (Italian: 5 è il numer...,"Section: Plot (1):\nIn the 1970s Naples, the r...",Section: Cast (1):\n\nSubsections (0):\n,None,None
2,A California Christmas,United States,ESX Entertainment,NaN,Lauren Swickard Josh Swickard,Lauren Swickard,106 minutes,Shaun Piccinino,NaN,NaN,A California Christmas is a 2020 Christmas fil...,Section: Plot (1):\nJoseph Van Aston lives a c...,Section: Cast (1):\n\nSubsections (0):\n,None,None


# Gestione dei titoli mancanti

In diz_mancanti uso come chiavi tutti i titoli mancanti e specifico se sono serie tv o film, successivamente aggiungo l'url dei titoli che ce l'hanno.

In [99]:
len(titoli_mancanti)

26

Creazione di __diz_mancanti__.

In [100]:
diz_mancanti = {}

for t in titoli_mancanti:
    diz_mancanti[t] = []

Qui metto per ogni chiave specifico se è una serie tv o un film, e successivamente metto a mano alcuni url.

In [101]:
for t in diz_mancanti:
    if tipo_titoli[t] == "TV":
        diz_mancanti[t].append("TV")
    else:
        diz_mancanti[t].append("Movie")

In [102]:
diz_mancanti['Il Traditore'].append("https://it.wikipedia.org/wiki/Il_traditore_(film_2019)")
diz_mancanti['Modalita aereo'].append("https://it.wikipedia.org/wiki/Modalit%C3%A0_aereo")
diz_mancanti['SanPa Sins of the Savior'].append("https://it.wikipedia.org/wiki/SanPa_-_Luci_e_tenebre_di_San_Patrignano")
diz_mancanti['My Hero Academia'].append("https://en.wikipedia.org/wiki/My_Hero_Academia")
diz_mancanti['The Champion'].append("https://it.wikipedia.org/wiki/Il_campione_(film_2019)")
diz_mancanti['Mister Felicita'].append("https://it.wikipedia.org/wiki/Mister_Felicit%C3%A0")
diz_mancanti["I'm Not a Killer"].append("https://it.wikipedia.org/wiki/Non_sono_un_assassino")
diz_mancanti['Il signor Diavolo'].append("https://it.wikipedia.org/wiki/Il_signor_Diavolo_(film)")
diz_mancanti['La prima pietra'].append("https://it.wikipedia.org/wiki/La_prima_pietra")
diz_mancanti['To Live'].append("https://it.wikipedia.org/wiki/Vivere_(film_2019)")

In [103]:
diz_mancanti

{'Animals on the Loose: A You vs. Wild Interactive Movie': ['Movie'],
 'Asphalt Burning': ['Movie'],
 'Biggie: I Got a Story to Tell': ['Movie'],
 'Coven': ['Movie'],
 "Don't Listen": ['Movie'],
 'Full Out 2: You Got This!': ['Movie'],
 'Grizzy & the Lemmings': ['TV'],
 "I'm Not a Killer": ['Movie',
  'https://it.wikipedia.org/wiki/Non_sono_un_assassino'],
 'Il Traditore': ['Movie',
  'https://it.wikipedia.org/wiki/Il_traditore_(film_2019)'],
 'Il signor Diavolo': ['Movie',
  'https://it.wikipedia.org/wiki/Il_signor_Diavolo_(film)'],
 'La prima pietra': ['Movie', 'https://it.wikipedia.org/wiki/La_prima_pietra'],
 'Midnight at the Magnolia': ['Movie'],
 'Mister Felicita': ['Movie',
  'https://it.wikipedia.org/wiki/Mister_Felicit%C3%A0'],
 'Modalita aereo': ['Movie',
  'https://it.wikipedia.org/wiki/Modalit%C3%A0_aereo'],
 'SanPa Sins of the Savior': ['TV',
  'https://it.wikipedia.org/wiki/SanPa_-_Luci_e_tenebre_di_San_Patrignano'],
 "So My Grandma's a Lesbian!": ['Movie'],
 'The Champio

Per trovare le informazioni sui titoli mancanti per alcuni faccio scraping, per altri procedo a mano.

Funzione ausiliaria che riempie il dizionario di turno.

In [104]:
def riempi_dati(df, Diz, lista_stringhe_da_cercare):

    limit = len(lista_stringhe_da_cercare)

    if (len(Diz)-1) != limit:
        print("Errore: non hai messo abbastanza stringhe!")
        
    else:
        i = 0
        
        for categoria in Diz:
            if categoria != "Titolo":
                aggiungi_valore(df, Diz, lista_stringhe_da_cercare[i], categoria)
                i = i +1

Creazione delle righe aggiuntive per i titoli mancanti

In [105]:
title_to_add = {}   # dizionario ausiliario che contiene per ogni titolo mancante la riga corrispondente 
                    # che aggiungerò al dataframe

Mancanti_serie = {'Titolo' : [], 'Paese' : [], 'Produzione' : [], 'Distribuzione' : [], 'Cast' : [], 
                    'Creatore' : [], 'Durata_episodio' : [], 'Genere' : [], 'N_stagioni' : [], 'N_episodi' : []}
Mancanti_film = {'Titolo' : [], 'Paese' : [], 'Produzione' : [], 'Distribuzione' : [], 'Cast' : [], 
                   'Creatore' : [], 'Durata' : [], 'Regista' : [], 'Budget' : [], 'Box_office' : []}

Per i titoli che hanno un url faccio scraping, prendendo le informazioni dalla tabella laterale.

In [106]:
for t in diz_mancanti:
    
    tipo = tipo_titoli[t]
    l = diz_mancanti[t]
    
    if tipo == "Movie":  # aggiungo il titolo al db dei film
        Mancanti_film['Titolo'].append(t)
    else: # aggiungo il titolo al db delle serie tv
        Mancanti_serie['Titolo'].append(t)    

    # Se ho un url faccio scraping
    if len(l) > 1:
        response = requests.get(l[1])
        html_page = response.text
        soup = BeautifulSoup(html_page, "html.parser")
        
        if t != "My Hero Academia":
            
            # Prendo le informazioni per film su wikipedia italiana
            if tipo == "Movie": # se ho un film
                
                tabella = soup.find("table", class_="sinottico") #film in wikipedia italiana
                if tabella:
                    df = pd.read_html(str(tabella), skiprows = 1)[0]   # df = dataframe temporaneo
                    df.columns = ["param", "title"]  
                    
                    riempi_dati(df, Mancanti_film, ['Paese di produzione', 'Casa di produzione', 
                                                    'Distribuzione in italiano', 'Cast', 'Soggetto',
                                                    'Durata', 'Regia', 'Budget', 'Box office'])
                    # Osservazione : nella wiki italiana non riesco a catturare così il cast
                    
            else: # se ho una serie tv
                tabella = soup.find("table", class_="sinottico")
                if tabella:
                    df = pd.read_html(str(tabella), skiprows = 1)[0]   # df = dataframe temporaneo
                    df.columns = ["param", "title"]
                    
                    riempi_dati(df, Mancanti_serie, ['Paese', 'Casa di produzione', 'Distributore', 
                                                     'Cast', 'Ideatore', 'Durata', 'Genere', 'Stagioni', 'Episodi'])

        # Per ora solo per My Hero Academia (e in generale per gli anime)
        else:
            tabella = soup.find("table", class_="infobox")
            if tabella:    
                df = pd.read_html(str(tabella), skiprows = 1)[0]   # df = dataframe temporaneo
                df.columns = ["param", "title"]  
                
                #Paese
                Mancanti_serie['Paese'].append('Japan')
                #Produzione
                aggiungi_valore(df,Mancanti_serie, 'Studio', 'Produzione')
                #Distribuzione
                aggiungi_valore(df,Mancanti_serie, 'Original network', 'Distribuzione')
                #Cast
                aggiungi_valore(df,Mancanti_serie, 'Cast', 'Cast')  # non ha senso il cast per un anime
                #Creatore
                aggiungi_valore(df,Mancanti_serie, 'Written\xa0by', 'Creatore') # per un anime il creatore è chi 
                                                                                    # ha creato il manga o chi 
                                                                                    # ha scritto la serie?
                #Durata_episodio
                aggiungi_valore(df,Mancanti_serie, 'Running time', 'Durata_episodio')  #non c'è la durata
                #Genere
                aggiungi_valore(df,Mancanti_serie, 'Genre', 'Genere')
                #N.stagioni  (gli anime non hanno le stagioni)
                aggiungi_valore(df,Mancanti_serie, 'No. of seasons', 'N_stagioni')
                #N.episodi
                aggiungi_valore(df,Mancanti_serie, 'Episodes', 'N_episodi')

    # Altrimenti ho dati mancanti e devo aggiungerli a mano
    else:
        if tipo == "Movie": 
            for categoria in Mancanti_film:   # aggiungo dati mancanti
                if categoria != "Titolo":
                    Mancanti_film[categoria].append("NaN")
        else:
            for categoria in Mancanti_serie:   # aggiungo dati mancanti
                if categoria != "Titolo":
                    Mancanti_serie[categoria].append("NaN")

Creo i dataframe con le righe aggiuntive da concatenare ai precedenti

In [107]:
aux_film = pd.DataFrame(data = Mancanti_film)
aux_serie = pd.DataFrame(data = Mancanti_serie)

A questi aggiungo __le colonne testuali__: faccio ancora scraping per resuperare altre informazioni dalla parte testuale della pagina wikipedia.

In [108]:
# Nuove colonne per df dei film
somm_f = []
trama_f = []
cast_testo_f = []
produzione_testo_f = []
critica_f = []

# Nuove colonne per df delle serie
somm_s = []
cast_testo_s = []
produzione_testo_s = []
critica_s = []

In [109]:
for t in diz_mancanti:
    # Film
    if tipo_titoli[t] == "Movie":
        l = diz_mancanti[t]
        
        if len(l) > 1:
            response = requests.get(l[1])
            html_page = response.text
            soup = BeautifulSoup(html_page, "html.parser")
            
            somm_f.append(soup.find("div", class_="mw-parser-output").p.text)
        
        else:
            somm_f.append("NaN")
        
        trama_f.append("NaN")
        cast_testo_f.append("NaN")
        produzione_testo_f.append("NaN")
        critica_f.append("NaN")
        
    # Serie tv
    else:
        l = diz_mancanti[t]
        
        if len(l) > 1:
            response = requests.get(l[1])
            html_page = response.text
            soup = BeautifulSoup(html_page, "html.parser")       
            
            somm_s.append(soup.find("div", class_="mw-parser-output").p.text)
            
        else:
            somm_s.append("NaN")
            
        cast_testo_s.append("NaN")
        produzione_testo_s.append("NaN")
        critica_s.append("NaN")

Aggiungo le nuove colonne ai dataframe ausiliari

In [110]:
# Film

aux_film['Sommario'] = somm_f
aux_film['Trama'] = trama_f
aux_film['Cast_testo'] = cast_testo_f
aux_film['Produzione_testo'] = produzione_testo_f 
aux_film['Critica'] = critica_f

In [114]:
aux_film.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata,Regista,Budget,Box_office,Sommario,Trama,Cast_testo,Produzione_testo,Critica
0,Animals on the Loose: A You vs. Wild Interacti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asphalt Burning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Biggie: I Got a Story to Tell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
aux_serie.head()

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata_episodio,Genere,N_stagioni,N_episodi
0,Grizzy & the Lemmings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SanPa Sins of the Savior,NaN,42,Netflix,NaN,Gianluca Neri,56-64 min (episodio),"docu-drama, drammatico",NaN,NaN
2,Demon Slayer: Kimetsu no Yaiba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,My Hero Academia,Japan,Bones,JNN (MBS) (#1–13)NNS (ytv) (#14–),NaN,Kōhei Horikoshi,NaN,"Adventure, fantasy,[1] superhero[2][3]",NaN,90 (List of episodes)


In [116]:
# Serie

aux_serie['Sommario'] = somm_s
aux_serie['Cast_testo'] = cast_testo_s
aux_serie['Produzione_testo'] = produzione_testo_s
aux_serie['Critica'] = critica_s

In [117]:
aux_film

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata,Regista,Budget,Box_office,Sommario,Trama,Cast_testo,Produzione_testo,Critica
0,Animals on the Loose: A You vs. Wild Interacti...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Asphalt Burning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Biggie: I Got a Story to Tell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Coven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Don't Listen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Full Out 2: You Got This!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,I'm Not a Killer,Italia,"Viola Film, Pepito Produzioni, Rai Cinema",01 Distribution,NaN,NaN,110 minuti,Andrea Zaccariello,NaN,NaN,Non sono un assassino è un film del 2019 diret...,NaN,NaN,NaN,NaN
7,Il Traditore,"Italia, Francia, Germania, Brasile","IBC Movie, Kavac Film, Rai Cinema",01 Distribution,NaN,Marco Bellocchio,148 min,Marco Bellocchio,NaN,NaN,Il traditore è un film del 2019 diretto da Mar...,NaN,NaN,NaN,NaN
8,Il signor Diavolo,Italia,"DueA Film, Rai Cinema",01 Distribution,NaN,NaN,86 minuti,Pupi Avati,NaN,NaN,Il signor Diavolo è un film del 2019 diretto d...,NaN,NaN,NaN,NaN
9,La prima pietra,Italia,"Fandango, Warner Bros. Entertainment Italia",Warner Bros. Italia,NaN,Stefano Massini,77 min,Rolando Ravello,NaN,NaN,Stefano Di Santi\n,NaN,NaN,NaN,NaN


Ora concateno ai due dataframe principali i due ausiliari appena costruiti.

In [118]:
serie_tv_wiki = pd.concat([df_serie_tv, aux_serie], ignore_index= True)

In [119]:
film_wiki = pd.concat([df_film, aux_film], ignore_index=True)

Ordino i dataframe per titolo.

In [120]:
serie_tv_wiki.sort_values(by = 'Titolo', inplace = True)

film_wiki.sort_values(by = 'Titolo', inplace = True)

# Dataframe risultanti

Sono __serie_tv_wiki__ e __film_wiki__.

In [121]:
serie_tv_wiki.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata_episodio,Genere,N_stagioni,N_episodi,Sommario,Cast_testo,Produzione_testo,Critica
0,Age of Samurai: Battle for Japan,NaN,Netflix,NaN,Masayoshi Haneda Masami Kosaka Hideaki Itō Hay...,NaN,43–45 minutes,NaN,1,6,Age of Samurai: Battle for Japan is an America...,None,None,None
1,Agents of S.H.I.E.L.D.,United States,ABC Studios Marvel Television Mutant Enemy Pro...,Walt Disney Television,Clark Gregg Ming-Na Wen Brett Dalton Chloe Ben...,Joss Whedon Jed Whedon Maurissa Tancharoen,41–44 minutes,Action Drama Science fiction Superhero,7,136 (list of episodes),Marvel's Agents of S.H.I.E.L.D. is an American...,Section: Cast and characters (1):\nClark Gregg...,Section: Production (1):\n\nSubsections (8):\n...,Section: Reception (1):\n\nSubsections (4):\nS...
2,Alice in Borderland,Japan,Robot Communications Inc.,Netflix,Kento Yamazaki Tao Tsuchiya Yūki Morinaga Keit...,NaN,41–52 minutes,Science fiction[1] Suspense-thriller[2] Drama[3],1,8 (list of episodes),"Alice in Borderland (Japanese: 今際の国のアリス, Hepbu...",Section: Cast and characters (1):\n\nSubsectio...,Section: Production (1):\n\nSubsections (3):\n...,Section: Reception (1):\n\nSubsections (2):\nS...


In [125]:
film_wiki.head(3)

,Titolo,Paese,Produzione,Distribuzione,Cast,Creatore,Durata,Regista,Budget,Box_office,Sommario,Trama,Cast_testo,Produzione_testo,Critica
0,365 Days,Poland,Ekipa Sp. z o.o. Future Space Next Film TVN,Next Film,Anna-Maria Sieklucka Michele Morrone Bronisław...,Tomasz Klimala Barbara Białowąs Tomasz Mandes ...,114 minutes,Barbara Białowąs Tomasz Mandes,NaN,$9.5 million[1],365 Days (Polish: 365 Dni) is a 2020 Polish er...,Section: Plot (1):\nAfter a meeting between th...,Section: Cast (1):\nMichele Morrone as Don Mas...,Section: Production (1):\nThe film scenes were...,Section: Reception (1):\nThe film made the top...
1,5 Is the Perfect Number,Italy,Rai Cinema,01 Distribution,Toni ServilloValeria GolinoCarlo BuccirossoIai...,Igort,100 minutes,Igort,NaN,NaN,5 Is the Perfect Number (Italian: 5 è il numer...,"Section: Plot (1):\nIn the 1970s Naples, the r...",Section: Cast (1):\n\nSubsections (0):\n,None,None
2,A California Christmas,United States,ESX Entertainment,NaN,Lauren Swickard Josh Swickard,Lauren Swickard,106 minutes,Shaun Piccinino,NaN,NaN,A California Christmas is a 2020 Christmas fil...,Section: Plot (1):\nJoseph Van Aston lives a c...,Section: Cast (1):\n\nSubsections (0):\n,None,None


### Esporto i dataframe risultanti in file csv.

In [123]:
serie_tv_wiki.to_csv("Dati_integrati/wiki_serie_dic-mar.csv")
film_wiki.to_csv("Dati_integrati/wiki_film_dic-mar.csv")